# Importing Libraries

In [ ]:
import random
import numpy as np
import pandas as pd
import networkx as nx
import statistics as st
from random import choice
from numpy.polynomial.polynomial import polyfit

# Defining Time Intervals

In [ ]:
years = list(range(2000, 2023 + 1))
years = [x * 100 for x in years]

In [ ]:
intervals = []

for year in years:
  for month in range(1, 12 + 1):
    if year == 202300 and month == 8:
      break
    inter = year + month
    intervals.append(inter)

In [ ]:
len(intervals)

283

In [ ]:
strIntervals = []

for interval in intervals:
  strInterval = str(interval)[0:4] + "-" + str(interval)[4:6]
  strIntervals.append(strInterval)

In [ ]:
G = nx.read_graphml("acquisitionNet.graphml")

# Calculating Temporal Features

In [ ]:
densities = []

for slot in intervals:
  selected_edges = [(u,v) for u,v,e in G.edges(data=True) if e['acquired_year_month'] <= slot]
  H = G.edge_subgraph(selected_edges)
  densities.append(nx.density(H))

In [ ]:
average_clusterings = []

for slot in intervals:
  selected_edges = [(u,v) for u,v,e in G.edges(data=True) if e['acquired_year_month'] <= slot]
  H = G.edge_subgraph(selected_edges)
  H = H.to_undirected()
  max_lcc = max(nx.connected_components(H), key=len)
  max_wcc = H.subgraph(max_lcc)
  clustering_coefficient = nx.average_clustering(max_wcc)
  average_clusterings.append(clustering_coefficient)

In [ ]:
def avg_short_path(N):
  H = N.to_undirected()
  max_lcc = max(nx.connected_components(H), key=len)
  max_wcc = H.subgraph(max_lcc)
  total = 0
  for i in range(1000):
    a = choice(list(max_wcc.nodes()))
    b = choice(list(max_wcc.nodes()))
    dist = nx.shortest_path_length(max_wcc, source=a, target=b, method='dijkstra')
    total += dist
  average_shortest_path_length = total / 1000
  return average_shortest_path_length

In [ ]:
path_lengths = []

for slot in intervals:
  selected_edges = [(u,v) for u,v,e in G.edges(data=True) if e['acquired_year_month'] <= slot]
  H = G.edge_subgraph(selected_edges)
  shotest = avg_short_path(H)
  path_lengths.append(shotest)

In [ ]:
component_counts = []

for slot in intervals:
  selected_edges = [(u,v) for u,v,e in G.edges(data=True) if e['acquired_year_month'] <= slot]
  H = G.edge_subgraph(selected_edges)
  component_count = list(nx.weakly_connected_components(H))
  component_counts.append(len(component_count))

In [ ]:
temporalData = pd.DataFrame(
    {'Interval': strIntervals,
     'Density': densities,
     'Average Clustering Coefficient': clustering_coefficient,
     'Average Shortest Path Length': path_lengths,
     'WCC Count': component_counts
    })

In [ ]:
temporalData.head()

,Interval,Density,Average Clustering Coefficient,Average Shortest Path Length,WCC Count
0,2000-01,0.000342,0.0,1.859,678
1,2000-02,0.000330,0.0,1.861,704
2,2000-03,0.000311,0.0,1.853,746
3,2000-04,0.000302,0.0,1.897,771
4,2000-05,0.000294,0.0,1.893,788


In [ ]:
temporalData.to_csv("temporalData.csv", index=False)